# Single Instrument Reference Data

This notebook demonstrates how to access and use the functinalities of **Instrument Reference** which are part of our **Yiedbook** module within LSEG Financial Analytics SDK.

The main goal of this notebook is to explore and display different approaches in fetching single Instrument reference by using and of 4 available methods:
1. Synchronous POST
2. Synchronous GET
3. Asynchronous POST
4. Asynchronous GET

## Imports

Import the following necessary modules:

- *request_bond_indic_sync* - main Synchronous POST method
- *request_bond_indic_sync_get* - main Synchronous GET method
- *request_bond_indic_async* - main Asynchronous POST method
- *request_bond_indic_async_get* - main Asynchronous GET method
- *IdentifierInfo* - Class wrapper for instrument identifiers
- *get_result* - method for obtaining results of Async method executions
- *json* - for display purposes 
- *time* - for response delay handling

In [1]:
from lseg_analytics.pricing.yield_book_rest import (
    request_bond_indic_sync,
    request_bond_indic_sync_get,
    request_bond_indic_async,
    request_bond_indic_async_get,
    IdentifierInfo,
    get_result
)

import json as js

import time

## Data Preparation

As all the methods consumes an input argument containing an identifier, here's how to prepare it:

In [2]:
# Select an ISIN or CUSIP ID of the instrument
identifier="NL0000102317"

# Prepare the input data container
instrument_input=[IdentifierInfo(identifier=identifier)]

## Synchronous POST Data retrieval

Once the data is prepared, it can be provided to the main method execution for data retrieval:

In [3]:
# Request bond indic with sync post
sync_post_response = request_bond_indic_sync(input=instrument_input)

## Synchronous GET Data retrieval

Opposed to POST method, GET only requires the identifier itself, as it runs in form of API body directly. 

In [4]:
# Request bond indic with sync get
sync_get_response = request_bond_indic_sync_get(id=identifier)

## Asynchronous POST Data retrieval

Main difference in Sync and Async approach is that execution of async methods leaves room for further code execution on user's side and the user can 'collect' the results whenever he needs them (if execution is finished by then).

So this approach has 2 steps:
1. Provide input data to execution method
2. Retrieve information by request_id received in step 1.

In [ ]:
# Request bond indic with async post
async_post_response = request_bond_indic_async(input=instrument_input)

attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_post_results_response = get_result(request_id_parameter=async_post_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + async_post_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{async_post_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

## Asynchronous GET Data retrieval

Difference between Sync and Async can be found in description above.

As Async execution is in queue-form it's not necessarily instantly finished. It can take time between submission of request and execution, and results are not in real-time.

Hence we present also the form of iterative results-request repetition to fetch the results as soon as they are ready.

**NOTE:** Alternative to this is wrapping the request in the job, and then using a method to check the job status.

In [ ]:
# Request bond indic with async get
async_get_response = request_bond_indic_async_get(id=identifier)

# Due to async nature, code Will perform the fetch 10 times, as result is not always ready instantly, with 3 second lapse between attempts
attempt = 1

while attempt < 10:

    from lseg_analytics.core.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_get_results_response = get_result(request_id_parameter=async_get_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + async_get_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{async_get_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

## Display results

Results data is structured in JSON format so it can be easily displayed using print function.

### Synchronous POST results

In [5]:
# Print results in json format
print(js.dumps(sync_post_response.as_dict(), indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-150645",
        "timeStamp": "2025-12-03T07:45:08Z",
        "responseType": "BOND_INDIC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "NL0000102317",
            "cusip": "63982A9P4",
            "indic": {
                "cfi": "DBFTFN",
                "ric": "NL10231=",
                "wkn": "230570",
                "figi": "BBG00005MTT8",
                "frtb": {
                    "sector": {
                        "value": "Sovereigns including central banks"
                    },
                    "riskWeight": 0.5,
                    "bucketNumber": 1,
                    "creditQuality": "IG",
                    "riskClassification": "CSR_NonSecuritised"
                },
                "isin": "NL0000102317",
                "cusip": "63982A9P4",
                "moody": [
                    {
                        "value": "Aaa",
                      

### Synchronous GET results

In [6]:
# Print results in json format
print(js.dumps(sync_get_response, indent=4))

{
    "data": {
        "isin": "NL0000102317",
        "cusip": "63982A9P4",
        "indic": {
            "cfi": "DBFTFN",
            "ric": "NL10231=",
            "wkn": "230570",
            "figi": "BBG00005MTT8",
            "frtb": {
                "sector": {
                    "value": "Sovereigns including central banks"
                },
                "riskWeight": 0.5,
                "bucketNumber": 1,
                "creditQuality": "IG",
                "riskClassification": "CSR_NonSecuritised"
            },
            "isin": "NL0000102317",
            "cusip": "63982A9P4",
            "moody": [
                {
                    "value": "Aaa",
                    "effectiveDate": "1998-05-05"
                }
            ],
            "permID": "15628455479",
            "source": "THOMREUT",
            "ticker": "NLGV",
            "country": "NL",
            "newIsin": "NL0000102317",
            "putFlag": false,
            "callFlag": false,


### Asynchronous POST results

In [9]:
# Print results in json format
print(js.dumps(async_post_results_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-1572773",
        "timeStamp": "2025-06-12T06:39:12Z",
        "responseType": "BOND_INDIC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "NL0000102317",
            "cusip": "63982A9P4",
            "indic": {
                "cfi": "DBFTFN",
                "ric": "NL10231=",
                "wkn": "230570",
                "figi": "BBG00005MTT8",
                "frtb": {
                    "sector": {
                        "value": "Sovereigns including central banks"
                    },
                    "riskWeight": 0.5,
                    "bucketNumber": 1,
                    "creditQuality": "IG",
                    "riskClassification": "CSR_NonSecuritised"
                },
                "isin": "NL0000102317",
                "cusip": "63982A9P4",
                "moody": [
                    {
                        "value": "Aaa",
                     

### Asynchronous GET results

In [10]:
# Print results in json format
print(js.dumps(async_get_results_response, indent=4))

{
    "data": {
        "isin": "NL0000102317",
        "cusip": "63982A9P4",
        "indic": {
            "cfi": "DBFTFN",
            "ric": "NL10231=",
            "wkn": "230570",
            "figi": "BBG00005MTT8",
            "frtb": {
                "sector": {
                    "value": "Sovereigns including central banks"
                },
                "riskWeight": 0.5,
                "bucketNumber": 1,
                "creditQuality": "IG",
                "riskClassification": "CSR_NonSecuritised"
            },
            "isin": "NL0000102317",
            "cusip": "63982A9P4",
            "moody": [
                {
                    "value": "Aaa",
                    "effectiveDate": "1998-05-05"
                }
            ],
            "permID": "15628455479",
            "source": "THOMREUT",
            "ticker": "NLGV",
            "country": "NL",
            "newIsin": "NL0000102317",
            "putFlag": false,
            "callFlag": false,
